In [ ]:
import pandas as pd
import pickle
from geopy.distance import geodesic

df_tiendas = pickle.load(open("tiendas.pkl", "rb"))
cosine_sim = pickle.load(open("tfidf_similarity.pkl", "rb"))

def obtener_recomendaciones_con_distancia(tienda_id, user_location, cosine_sim=cosine_sim, radius_km=1):
    if tienda_id not in df_tiendas['tienda_id'].values:
        return []  # O puedes devolver un mensaje de error adecuado
    
    idx = df_tiendas.index[df_tiendas['tienda_id'] == tienda_id].tolist()[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:5]
    tienda_indices = [i[0] for i in sim_scores]
    
    recomendaciones = []
    for i in tienda_indices:
        tienda_location = (df_tiendas.iloc[i]['latitud'], df_tiendas.iloc[i]['longitud'])
        distance = geodesic(user_location, tienda_location).km
        if distance <= radius_km:
            tienda_info = df_tiendas.iloc[i].to_dict()
            tienda_info['distance'] = round(distance, 2)
            recomendaciones.append(tienda_info)
    return recomendaciones

tienda_id = 19321  # ID de la tienda
user_location = (-12.130843, -76.982489)  # Ubicación del usuario (para probar puse ubicacion de la tienda)
radius_km = 5  # Radio de búsqueda en kilómetros
print(f'Tienda: {tienda_id}, Ubicación: {user_location}, Radio: {radius_km}')
recomendaciones = obtener_recomendaciones_con_distancia(tienda_id, user_location, radius_km=radius_km)  # Aumenta el radio para depuración
print(recomendaciones)

Tienda: 19321, Ubicación: (-12.130843, -76.982489), Radio: 5
[{'tienda_id': 17222, 'nombre': 'Villa Chicken', 'categorias': 'restaurant, food', 'latitud': -12.14026, 'longitud': -76.9988566, 'direccion': 'Doña Rosa 144, Santiago de Surco', 'calificaciones': 4.3, 'rango_precios': 'Alto', 'metodos_pago': 'Transferencia', 'tamano_tienda': 'Mediana', 'estacionamiento': 'No', 'horario_apertura': 'Horario laboral', 'distrito': 'Santiago de Surco', 'zona': 'Lima Sur', 'combined_features': 'Villa Chicken restaurant, food Alto Transferencia Mediana No Horario laboral Lima Sur', 'distance': 2.063794580165563}, {'tienda_id': 18733, 'nombre': 'VIA CHICKEN', 'categorias': 'restaurant, food', 'latitud': -12.1351972, 'longitud': -76.9415649, 'direccion': 'Avenida Jose Carlos Mariategui 3536, Villa María del Triunfo', 'calificaciones': 4.0, 'rango_precios': 'Medio', 'metodos_pago': 'Efectivo', 'tamano_tienda': 'Mediana', 'estacionamiento': 'No', 'horario_apertura': '24 horas', 'distrito': 'Ate', 'zona